```{currentmodule} optimap
```

In [ ]:
# Code snippet for rendering animations in the docs
from IPython.display import HTML
import warnings
import matplotlib
matplotlib.rcParams['animation.embed_limit'] = 2**128

def render_ani_func(f):
    om.utils.disable_interactive_backend_switching()
    plt.switch_backend('Agg')
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        ani = f()
    %matplotlib inline
    om.utils.enable_interactive_backend_switching()

    vid = HTML(ani.to_html5_video())
    plt.close('all')
    return vid

```{tip}
Download this tutorial as a {download}`Jupyter notebook <converted/basics.ipynb>`, or a {download}`python script <converted/basics.py>` with code cells. We highly recommend using [Visual Studio Code](#vscode) to execute this tutorial. Alternatively, you could run the Python script in a terminal with ``python3 basics.py`` from the folder where the file is located.
```

# Tutorial 1: Basics

This tutorial will walk you through the basics of using ``optimap``. 

``optimap`` is a package that can be imported and used in Python scripts. [Python](https://en.wikipedia.org/wiki/Python_programming_language) is a freely available programming language that is also very popular in scientific computing. The simplest way to access the functionality of ``optimap`` is to import it at the beginning of a new Python script as follows:

In [ ]:
import optimap as om

When running this command and ``optimap`` was installed correctly, there should be no further output. We now have access to all the functions in the `optimap` package, and it can then be accessed by typing `om.` followed by a specific function as shown below (e.g. `om.load_video()`). If the import produces an error then ``optimap`` was  not installed correctly, see [Installation Guide](#installing) for further details. 

``optimap`` relies heavily on other open-source software packages and libraries, foremost [NumPy](https://numpy.org/), which is a numerical programming library, [Matplotlib](https://matplotlib.org/), which is a library for plotting data, and [OpenCV](https://opencv.org/), which is a popular library for computer vision. In this example, we will import ``numpy`` and ``matplotlib`` right after ``optimap`` as follows:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

You do not necessarily have to import `numpy` and `matplotlib` in your analysis scripts as it will be imported internally by `optimap`. Here we imported these packages to be able to work directly with some of their functions in our script and write some custom code for analysis and plotting.

If you encounter any issues with the steps above please see the [Installation Guide](https://optimap.readthedocs.io/en/latest/chapters/getting_started/).

## Loading a video file

The following file formats are currently supported by ``optimap``:

* .tif, .tiff (TIFF stack)
* .dat (MultiRecorder)
* .gsd, .gsh (SciMedia MiCAM 05)
* .rsh, .rsm, .rsd (SciMedia MiCAM ULTIMA)
* .npy (numpy array)
* .mat (MATLAB), loads the first field in the file

We can use the `optimap.load_video` function to load a video file. Here, we will first download an example file from our website [cardiacvision.ucsf.edu](https://cardiacvision.ucsf.edu) and then load it as a video. Alternatively, you could load your own file by replacing `filepath` with the filename of your video. After loading, the video corresponds to a three-dimensional `NumPy`-array where the first dimension is time and the other two dimensions are the x- and y-dimensions, respectively. The example file shows a fibrillating, weakly contracting rabbit heart stained with voltage-sensitive dye (Di-4-ANEPPS) imaged using a Basler acA720-520um camera at 500fps. Due to the staining, the action potential wave is inverted, i.e. an upstroke is observed as a negative deflection. In our case, we extracted a short section of the original recording and saved the raw data as a numpy file (`.npy`). The data is from Chowdhary et al. {footcite:t}`Chowdhary2023`. Experimenters: Jan Lebert, Shrey Chowdhary & Jan Christoph (University of California, San Francisco, USA), 2023.

In [ ]:
filepath = om.utils.retrieve_example_data('Example_02_VF_Rabbit_Di-4-ANEPPS_Basler_acA720-520um.npy')
video = om.load_video(filepath)

om.print_properties(video)

`optimap` imports video data as three-dimensional numpy arrays with shape (Time, Height, Width). This convention is used throughout the library. The function `optimap.print_properties()` displays the dimensions and maximal and minimal intensity values of a video. Our example file has 1040 video frames. See {func}`load_video` for additional arguments, e.g. to load only a subset of the frames or to use memory mapping to reduce memory usage.

```python
video = om.load_video('Example.dat', start_frame=100, frames=1000, step=2, use_mmap=True)
```

For some file formats, optimap can also load the corresponding metadata using {func}`load_metadata`. For example, the following code loads the metadata of a MiCAM ULTIMA recording:

```python
metadata = om.load_metadata('Example.rsh')
```

To crop, rotate, or flip a video see {mod}`optimap.video` for a list of available functions.

## Playing videos
Videos can be viewed using either:
1. the built-in viewer {func}`play_video` based on matplotlib
2. Monochrome, which is a more advanced and performant viewer with better interactive features

### Using the built-in viewer

In [ ]:
om.play_video(video);

In [ ]:
video = video[:1000]
render_ani_func(lambda: om.video.play(video, interval=15))

See API documentation for {func}`play_video` for a list of available arguments. For instanxce, it is possible to specify a title, a range (vmin=0, vmax=1) and colormap:
```python
om.play_video(video, title='Example video', vmin=0, vmax=1, cmap='gray', interval=20)
```



### Using Monochrome
Monochrome needs to be installed separately. We will provide installation instructions soon.

In [ ]:
import monochrome as mc
mc.show(video, "raw video")

See the {doc}`monochrome` documentation and {func}`monochrome.show` for more information. For example, click in the video to view time traces at the selected positions.

## Viewing and extracting traces

Using `optimap` it is possible to quickly extract and display optical traces from any location in a video and display them using the built-in player. With the {func}`select_traces` it is possible to interactively select, extract and view an optical trace. Click on the video image on the left to select a single position or multiple positions. Right click to remove positions. Close the window to continue.

In [ ]:
traces, positions = om.select_traces(video, size=5, fps=500)

In [ ]:
positions = [(127, 147), (130, 209), (202, 136)]

fig, axs = plt.subplots(1,2, figsize=(11,4))
om.trace.show_positions(video[0], positions, ax=axs[0])
traces = om.trace.extract_traces(video, positions, size=5, ax=axs[1], show=True, fps=500)
plt.show()

Traces can be extracted from a single pixel or from a small window surrounding the pixel. The `size` parameter controls the dimensions of the window. By default, this window is a rectangle with dimensions `(size, size)`, but it can also be set to `'disc'` using the `window` parameter, which then sets the window to a circular region with diameter `size` around the position. The default size of the window is 5 by 5 pixels (rectangular) or a diameter of 5 pixels (disc). To get the exact pixel values without spatial averaging, set `size=1`. **Note that the traces above include strong motion artifacts.** If you would like to display the time axis in seconds rather than frames, use the `fps` (frames per second) parameter.

In [ ]:
traces = om.extract_traces(video, positions, size=1, show=True, fps=500)

See {func}`select_traces` for more information and {func}`optimap.trace.set_default_trace_window` to change the default window type.

Internally {func}`extract_traces` uses {func}`show_traces` to plot traces. In general, all plotting functions in optimap have an `ax` parameter which can be used to specify a custom matplotlib axes object. For example, we can create a figure with two subplots and show the positions on the first subplot and the traces on the second subplot with milliseconds as time unit:

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(10,5))

om.trace.show_positions(video[0], positions, ax=axs[0])

x_axis_ms = (np.arange(video.shape[0]) / 500.0) * 1000
traces = om.extract_traces(video[:300],
                           positions,
                           x=x_axis_ms[:300],
                           size=5,
                           window='disc',
                           ax=axs[1],
                           show=True)
axs[1].set_xlabel('Time [ms]')
plt.show()

Since the optical traces above were sampled from a raw optical mapping video showing a fibrillating contracting heart (no Blebbistatin was used during the experiment), the traces contain strong motion artifacts. We will now optimap to track and stabilize motion to compensate these motion artifacts.

## Motion Compensation

`optimap` can be used to track motion and compensate motion artifacts in optical mapping recordings. The fibrillating heart from our example contracts rapidly and slightly moves in the recording. Even though the motion is small, it can have a strong effect on the optical traces in the form of motion artifacts and prevent further analysis. We can use the {func}`motion_compensate` function to compensate the motion:

In [ ]:
warped = om.motion_compensate(video,
                              contrast_kernel=5,
                              presmooth_spatial=1,
                              presmooth_temporal=1)

The details of the motion tracking and compensation routines are described in {cite}`Christoph2018a` and {cite}`Lebert2022`. See [](motion_compensation) for more detailed information and further examples. Let's view the original video and the motion-compensated video side by side using {func}`optimap.video.play2`:

In [ ]:
om.video.play2(video,
               warped,
               title1="with motion",
               title2="without motion",
               skip_frame=3);

In [ ]:
render_ani_func(lambda: om.video.play2(video, warped, title1="with motion", title2="without motion", skip_frame=1))

## Saving and rendering videos

Let's save the motion-compensated recording as a tiff stack and also render it to a .mp4 video file.

In [ ]:
om.video.save_video(warped, 'warped_recording.tiff')
om.video.export_video(warped, 'warped_recording.mp4', fps=50)

The optimap video-player functions such as {func}`optimap.video.play2` can also be exported to a video file:
```python
animation = om.video.play2(video, warped, title1='Raw', title2='Compensated')
animation.save('Example.mp4')
```
See {meth}`matplotlib.animation.Animation.save` for more details.

## Fluorescence wave isolation

```{note}
The rest of this tutorial is currently work in progress. We will add more information soon.
```

### Sliding window normalization

To better visualize the action potential propagation we can compute a pixel-wise normalization to [0, 1] using a sliding/rolling window. This is done using the {func}`video.normalize_pixelwise_slidingwindow` function. The `window_size` parameter controls the size of the sliding window, here we use a window size of 60 frames (120 ms).

In [ ]:
norm_raw = om.video.normalize_pixelwise_slidingwindow(video, window_size=60)
norm_warped = om.video.normalize_pixelwise_slidingwindow(warped, window_size=60)
om.video.play2(norm_raw, norm_warped, title1="with motion", title2="without motion");

In [ ]:
norm_raw = om.video.normalize_pixelwise_slidingwindow(video, window_size=60)
norm_warped = om.video.normalize_pixelwise_slidingwindow(warped, window_size=60)
def f():
    return om.video.play2(norm_raw[:500], norm_warped[:500], title1="with motion", title2="without motion", interval=20)
render_ani_func(f)

# docs job run of memory, try to free some
import gc
del video, norm_raw
gc.collect()

The dark waves is the fluorescent signal, they correspond to the action potential propagation.

In [ ]:
mask = om.background_mask(warped[0])
norm_warped[:, mask] = 1.0

In [ ]:
om.video.play(norm_warped, interval=20);

In [ ]:
render_ani_func(lambda: om.video.play(norm_warped, interval=40))

In [ ]:
alpha = np.clip(-(2*norm_warped - 1), 0, 1)
om.video.play_with_overlay(warped, 1-norm_warped, alpha=alpha);

In [ ]:
alpha = np.clip(-(2*norm_warped - 1), 0, 1)
render_ani_func(lambda: om.video.play_with_overlay(warped[:500], (1-norm_warped)[:500], alpha=alpha[:500], interval=20))
del alpha, norm_warped
gc.collect()

### Temporal difference

In [ ]:
diff = om.video.temporal_difference(warped, 10)
diff[diff > 0] = 0
diff[:, mask] = 0
diff = om.video.normalize_pixelwise_slidingwindow(-diff, window_size=60)
om.video.play_with_overlay(warped, diff, vmin_overlay=-1, vmax_overlay=1)

In [ ]:
diff = om.video.temporal_difference(warped[:500], 10)
diff[diff > 0] = 0
diff[:, mask] = 0
diff = om.video.normalize_pixelwise_slidingwindow(-diff, window_size=60)
render_ani_func(lambda: om.video.play_with_overlay(warped[:500], diff, vmin_overlay=-1, vmax_overlay=1, interval=20))
del diff